### Applied Data Science Capstone Week 3 - 3

In [1]:
from bs4 import BeautifulSoup
import pandas as pd

import numpy as np # library to handle data in a vectorized manner


pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')


Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
geopy                     1.18.1                     py_0    conda-forge
Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
folium                    0.5.0                      py_0    conda-forge
Libraries imported.


Download data.

In [2]:
!wget -q -O 'List_of_postal_codes_of_Canada.html' https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M
# postalcode - geographical coordinates
!wget -q -O 'geopostal.csv' https://cocl.us/Geospatial_data
    
!wget -q -O 'tront.geojson' https://geo.nyu.edu/download/file/nyu-2451-36180-geojson.json

Read a downloaded geojson.

In [3]:
with open('tront.geojson') as json_data:
    tront_geojson = json.load(json_data)
#newyork_data

Read rows in file that was downloaded from http://cocl.us/Geospatial_data except for the top row.

In [4]:
import csv
postal_to_geographica_coordinates = {}
with open('geopostal.csv') as csvfile:
    csvreader = csv.reader(csvfile, delimiter=',')
    
    for index, row in enumerate(csvreader):
        if index > 0 :  postal_to_geographica_coordinates.setdefault(row[0], (row[1],row[2]))
#postal_to_geographica_coordinates            

Function tp get text in table's cell is in &lt;td&gt;abc&lt;/td&gt; or &lt;td&gt;&lt;a&gt;abc&lt;/a&gt;&lt;/td&gt;.

In [5]:
def get_text_in_tag(tag):
    """
    get text in like <td>aaa</td> or <td><a>aaa</a></td> from argument tag.
    """
    if tag.string is not None:
        return tag.string
    else:
        return tag.find('a').string

                             

Load a downloaded wiki page.

In [6]:
with open('List_of_postal_codes_of_Canada.html') as html_file:
    soup = BeautifulSoup(html_file,"lxml")

Read the 2nd or later rows in table and create dictionary.<br/>
When postalcode as key is in dictionary alaready, add Neighbourhood to array in value of dictionary.

In [7]:
not_assigned = 'Not assigned'
table_header=[ 'Neighborhood', 'PostalCode', 'Borough', 'Latitude', 'Longitude', 'IsTront']

table_values = []
postcode_dict = {} 

table_tag = soup.table
first_row = table_tag.find('tr')

for tr in first_row.find_next_siblings():
    row = [ get_text_in_tag(td) if td.string is None else td.string.strip() for td in tr.find_all('td')]

    if row[1] == not_assigned:
        continue
    elif row[2] == not_assigned:
        postcode_dict.setdefault(row[1],  [row[0], row[1], postal_to_geographica_coordinates[row[0]], row[1].find('Tront') > -1  ])
    else:
        postcode_dict.setdefault(row[2],  [row[0], row[1], postal_to_geographica_coordinates[row[0]], row[1].find('Toronto') > -1  ])
        
data_src = [ [key, value[0], value[1], float(value[2][0]), float(value[2][1]), value[3] ] for key,value in postcode_dict.items()]     

Create a DataFrame.

In [8]:
neighborhoods = pd.DataFrame(data_src, columns=table_header)
print(neighborhoods.shape)
neighborhoods.head()

(210, 6)


,Neighborhood,PostalCode,Borough,Latitude,Longitude,IsTront
0,West Deane Park,M9B,Etobicoke,43.650943,-79.554724,False
1,Regent Park,M5A,Downtown Toronto,43.654260,-79.360636,True
2,Sullivan,M1T,Scarborough,43.781638,-79.304302,False
3,Victoria Hotel,M5L,Downtown Toronto,43.648198,-79.379817,True
4,L'Amoreaux West,M1W,Scarborough,43.799525,-79.318389,False


Get the geograpical coordinate of Tronto city. 

In [9]:
address = 'Toronto city'

geolocator = Nominatim()
location = geolocator.geocode(address)
toronto_latitude = location.latitude
toronto_longitude = location.longitude
print('The geograpical coordinate of Toronto City are {}, {}.'.format(toronto_latitude, toronto_longitude))

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  app.launch_new_instance()


The geograpical coordinate of Toronto City are 43.653963, -79.387207.


In [10]:
toronto_data = neighborhoods.loc[neighborhoods['IsTront'] == True]
toronto_data.head()

,Neighborhood,PostalCode,Borough,Latitude,Longitude,IsTront
1,Regent Park,M5A,Downtown Toronto,43.654260,-79.360636,True
3,Victoria Hotel,M5L,Downtown Toronto,43.648198,-79.379817,True
9,Davisville,M4S,Central Toronto,43.704324,-79.388790,True
18,North Midtown,M5R,Central Toronto,43.672710,-79.405678,True
28,Trinity,M6J,West Toronto,43.647927,-79.419750,True


Show neighborhoods of Toronto city

In [11]:
# create map of Manhattan using latitude and longitude values
map_toronto = folium.Map(location=[toronto_latitude, toronto_longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Neighborhood']):
    label = toronto_data['PostalCode']
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [12]:
# @hidden_cell
CLIENT_ID = 'SCXVI3PF4XNWFUT0FM54TC0GTSS1JHEQ3PT500M421AGHWH5' # your Foursquare ID
CLIENT_SECRET = 'RVTD4SO1HUNBLGNWN04KSU1WKRTDXMB41ELHKT5V4YDKMPEO' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

LIMIT=100

The function to repeat the same process to all the neighborhood

In [13]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Get venues in Tronto via foursquare API.

In [14]:
toronto_venues = getNearbyVenues( names = toronto_data['Neighborhood'],
                                   latitudes = toronto_data['Latitude'],
                                   longitudes = toronto_data['Longitude']
                                  )

Regent Park
Victoria Hotel
Davisville
North Midtown
Trinity
Island airport
Grange Park
The Annex
Commerce Court
Railway Lands
Adelaide
Exhibition Place
Toronto Islands
Little Portugal
Forest Hill North
Harbord
CN Tower
St. James Town
Underground city
Rathnelly
Christie
Central Bay Street
Swansea
Church and Wellesley
Forest Hill SE
University of Toronto
The Beaches
King and Spadina
Ryerson
Parkdale Village
Garden District
High Park
Stn A PO Boxes 25 The Esplanade
Cabbagetown
Brockton
Richmond
King
The Junction South
The Beaches West
Dufferin
Toronto Dominion Centre
Harbourfront East
Chinatown
Deer Park
Harbourfront
Studio District
Parkdale
Berczy Park
South Hill
First Canadian Place
Lawrence Park
Summerhill East
Harbourfront West
Business reply mail Processing Centre969 Eastern
Yorkville
Union Station
Davisville North
South Niagara
Design Exchange
Summerhill West
Rosedale
The Danforth West
Forest Hill West
Roncesvalles
Riverdale
Moore Park
Dovercourt Village
Bathurst Quay
Roselawn
Kensi

In [15]:
print(toronto_venues.shape)
toronto_venues.head()

(3181, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Regent Park,43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,Regent Park,43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,Regent Park,43.65426,-79.360636,Toronto Cooper Koo Family Cherry St YMCA Centre,43.653191,-79.357947,Gym / Fitness Center
3,Regent Park,43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,Regent Park,43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot


Check how many venues were returned for each neighborhood

In [16]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Adelaide,100,100,100,100,100,100
Bathurst Quay,12,12,12,12,12,12
Berczy Park,55,55,55,55,55,55
Brockton,19,19,19,19,19,19
Business reply mail Processing Centre969 Eastern,18,18,18,18,18,18
CN Tower,12,12,12,12,12,12
Cabbagetown,48,48,48,48,48,48
Central Bay Street,80,80,80,80,80,80
Chinatown,98,98,98,98,98,98


Let's find out how many unique categories can be curated from all the returned venues

In [17]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 232 uniques categories.


Analyze Each Neighborhood

In [18]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Piano Bar,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Regent Park,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

examine the new dataframe size.

In [19]:
toronto_onehot.shape

(3181, 232)

group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [20]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Store,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Piano Bar,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wings Joint,Women's Store
0,Adelaide,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.040000,0.000000,0.00,0.000000,0.010000,0.0100,0.000000,0.030000,0.0000,0.000000,0.000000,0.000000,0.000000,0.030000,0.000000,0.030000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020000,0.01,0.020000,0.000000,0.000000,0.01,0.020000,0.010000,0.0,0.000000,0.050000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.030000,0.010000,0.060000,0.000000,0.000000,0.00,0.000000,0.000000,0.020000,0.000000,0.030000,0.000,0.000000,0.000000,0.000000,0.000000,0.010000,0.0100,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.010000,0.00

show shape

In [21]:
toronto_grouped.shape

(72, 232)

print each neighborhood along with the top 5 most common venues

In [22]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide----
                 venue  freq
0          Coffee Shop  0.06
1                 Café  0.05
2      Thai Restaurant  0.04
3           Steakhouse  0.04
4  American Restaurant  0.04


----Bathurst Quay----
              venue  freq
0    Airport Lounge  0.17
1   Airport Service  0.17
2  Airport Terminal  0.17
3  Sculpture Garden  0.08
4           Airport  0.08


----Berczy Park----
                venue  freq
0         Coffee Shop  0.07
1        Cocktail Bar  0.05
2          Restaurant  0.05
3  Italian Restaurant  0.04
4              Bakery  0.04


----Brockton----
                   venue  freq
0                   Café  0.11
1  Performing Arts Venue  0.11
2         Breakfast Spot  0.11
3            Coffee Shop  0.11
4      Convenience Store  0.05


----Business reply mail Processing Centre969 Eastern----
                venue  freq
0  Light Rail Station  0.11
1         Yoga Studio  0.06
2       Auto Workshop  0.06
3                Park  0.06
4          Comic Shop  0.06


----C

              venue  freq
0    Airport Lounge  0.17
1   Airport Service  0.17
2  Airport Terminal  0.17
3  Sculpture Garden  0.08
4           Airport  0.08


----Rathnelly----
              venue  freq
0       Coffee Shop  0.13
1               Pub  0.13
2       Pizza Place  0.07
3        Bagel Shop  0.07
4  Sushi Restaurant  0.07


----Regent Park----
         venue  freq
0  Coffee Shop  0.16
1          Pub  0.06
2       Bakery  0.06
3         Café  0.06
4         Park  0.06


----Richmond----
                 venue  freq
0          Coffee Shop  0.06
1                 Café  0.05
2      Thai Restaurant  0.04
3           Steakhouse  0.04
4  American Restaurant  0.04


----Riverdale----
                venue  freq
0    Greek Restaurant  0.24
1         Coffee Shop  0.10
2      Ice Cream Shop  0.07
3  Italian Restaurant  0.05
4           Bookstore  0.05


----Roncesvalles----
                         venue  freq
0               Breakfast Spot  0.12
1                    Gift Shop  0.12
2  Ea

Write a function to sort the venues in descending order.

In [23]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

reate the new dataframe and display the top 10 venues for each neighborhood.

In [24]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adelaide,Coffee Shop,Café,American Restaurant,Steakhouse,Thai Restaurant,Gym,Hotel,Cosmetics Shop,Restaurant,Asian Restaurant
1,Bathurst Quay,Airport Terminal,Airport Service,Airport Lounge,Harbor / Marina,Airport Gate,Sculpture Garden,Boat or Ferry,Airport Food Court,Airport,Donut Shop
2,Berczy Park,Coffee Shop,Cocktail Bar,Restaurant,Bakery,Steakhouse,Seafood Restaurant,Farmers Market,Café,Cheese Shop,Pub
3,Brockton,Coffee Shop,Café,Breakfast Spot,Performing Arts Venue,Gym,Furniture / Home Store,Pet Store,Climbing Gym,Caribbean Restaurant,Burrito Place
4,Business reply mail Processing Centre969 Eastern,Light Rail Station,Yoga Studio,Auto Workshop,Smoke Shop,Park,Farmers Market,Fast Food Restaurant,Spa,Brewery,Burrito Place
5,CN Tower,Airport Terminal,Airport Service,Airport Lounge,Harbor / Marina,Airport Gate,Sculpture Garden,Boat or Ferry,Airport Food Court,Airport,Donut Shop
6,Cabbagetown,Coffee Shop,Restaurant,Bakery,Indian Restaurant,Pizza Place,Italian Restaurant,Pub,Café,Gift Shop,Butcher
7,Central Bay Street,Coffee Shop,Café,Italian Restaurant,Bar,Ice Cream Shop,Burger Joint,Sandwich Place,Salad Place,Bubble Tea Shop,Falafel Restaurant
8,Chinatown,Café,Bar,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Bakery,Mexican Restaurant,Coffee Shop,Dumpling Restaurant,Chinese Restaurant,Dessert Shop
9,Christie,Grocery Store,Café,Park,Convenience Store,Nightclub,Baby Store,Diner,Athletics & Sports,Restaurant,Italian Restaurant


# Crustering

Run k-means to cluster the neighborhood into 5 clusters.

In [25]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:20] 

array([2, 1, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2], dtype=int32)

Create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [26]:
toronto_merged = toronto_data

# add clustering labels
toronto_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Neighborhood,PostalCode,Borough,Latitude,Longitude,IsTront,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Regent Park,M5A,Downtown Toronto,43.654260,-79.360636,True,2,Coffee Shop,Park,Bakery,Café,Pub,Breakfast Spot,Restaurant,Theater,Mexican Restaurant,Health Food Store
3,Victoria Hotel,M5L,Downtown Toronto,43.648198,-79.379817,True,1,Coffee Shop,Café,Hotel,Restaurant,American Restaurant,Bakery,Deli / Bodega,Seafood Restaurant,Steakhouse,Gym
9,Davisville,M4S,Central Toronto,43.704324,-79.388790,True,2,Dessert Shop,Sandwich Place,Café,Italian Restaurant,Coffee Shop,Pizza Place,Seafood Restaurant,Sushi Restaurant,Burger Joint,Japanese Restaurant
18,North Midtown,M5R,Central Toronto,43.672710,-79.405678,True,2,Coffee Shop,Café,Sandwich Place,Pizza Place,Pharmacy,Indian Restaurant,Cosmetics Shop,Pub,Burger Joint,Liquor Store
28,Trinity,M6J,West Toronto,43.647927,-79.419750,True,2,Bar,Men's Store,Restaurant,Coffee Shop,Asian Restaurant,Café,Pizza Place,Cocktail Bar,Vietnamese Restaurant,Record Shop


 visualize the resulting clusters

In [27]:
# create map
map_clusters = folium.Map(location=[toronto_latitude, toronto_longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# Examine Clusters

Cluster 1

In [28]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[0] + list(range(7, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
150,First Canadian Place,Café,Coffee Shop,Hotel,Restaurant,American Restaurant,Steakhouse,Gym,Gastropub,Bakery,Seafood Restaurant


Cluster 2

In [29]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[0] + list(range(7, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Victoria Hotel,Coffee Shop,Café,Hotel,Restaurant,American Restaurant,Bakery,Deli / Bodega,Seafood Restaurant,Steakhouse,Gym
31,Island airport,Airport Terminal,Airport Service,Airport Lounge,Harbor / Marina,Airport Gate,Sculpture Garden,Boat or Ferry,Airport Food Court,Airport,Donut Shop
97,Ryerson,Clothing Store,Coffee Shop,Café,Cosmetics Shop,Middle Eastern Restaurant,Italian Restaurant,Restaurant,Plaza,Diner,Pizza Place
105,High Park,Café,Mexican Restaurant,Bar,Grocery Store,Arts & Crafts Store,Music Venue,Diner,Cajun / Creole Restaurant,Sandwich Place,Bookstore
110,Brockton,Coffee Shop,Café,Breakfast Spot,Performing Arts Venue,Gym,Furniture / Home Store,Pet Store,Climbing Gym,Caribbean Restaurant,Burrito Place
127,Chinatown,Café,Bar,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Bakery,Mexican Restaurant,Coffee Shop,Dumpling Restaurant,Chinese Restaurant,Dessert Shop
162,Harbourfront West,Airport Terminal,Airport Service,Airport Lounge,Harbor / Marina,Airport Gate,Sculpture Garden,Boat or Ferry,Airport Food Court,Airport,Donut Shop


Cluster 3

In [30]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[0] + list(range(7, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Regent Park,Coffee Shop,Park,Bakery,Café,Pub,Breakfast Spot,Restaurant,Theater,Mexican Restaurant,Health Food Store
9,Davisville,Dessert Shop,Sandwich Place,Café,Italian Restaurant,Coffee Shop,Pizza Place,Seafood Restaurant,Sushi Restaurant,Burger Joint,Japanese Restaurant
18,North Midtown,Coffee Shop,Café,Sandwich Place,Pizza Place,Pharmacy,Indian Restaurant,Cosmetics Shop,Pub,Burger Joint,Liquor Store
28,Trinity,Bar,Men's Store,Restaurant,Coffee Shop,Asian Restaurant,Café,Pizza Place,Cocktail Bar,Vietnamese Restaurant,Record Shop
36,Grange Park,Café,Bar,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Bakery,Mexican Restaurant,Coffee Shop,Dumpling Restaurant,Chinese Restaurant,Dessert Shop
39,The Annex,Coffee Shop,Café,Sandwich Place,Pizza Place,Pharmacy,Indian Restaurant,Cosmetics Shop,Pub,Burger Joint,Liquor Store
40,Commerce Court,Coffee Shop,Café,Hotel,Restaurant,American Restaurant,Bakery,Deli / Bodega,Seafood Restaurant,Steakhouse,Gym
41,Railway Lands,Airport Terminal,Airport Service,Airport Lounge,Harbor / Marina,Airport Gate,Sculpture Garden,Boat or Ferry,Airport Food Court,Airport,Donut Shop
47,Adelaide,Coffee Shop,Café,American Restaurant,Steakhouse,Thai Restaurant,Gym,Hotel,Cosmetics Shop,Restaurant,Asian Restaurant
53,Exhibition Place,Coffee Shop,Café,Breakfast Spot,Performing Arts Venue,Gym,Furniture / Home Store,Pet Store,Climbing Gym,Caribbean Restaurant,Burrito Place


Cluster 4

In [31]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[0] + list(range(7, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
80,Christie,Grocery Store,Café,Park,Convenience Store,Nightclub,Baby Store,Diner,Athletics & Sports,Restaurant,Italian Restaurant
82,Swansea,Café,Coffee Shop,Pizza Place,Sushi Restaurant,Bookstore,Italian Restaurant,Indie Movie Theater,Food,Baby Store,Diner
142,South Hill,Pub,Coffee Shop,American Restaurant,Supermarket,Sushi Restaurant,Bagel Shop,Fried Chicken Joint,Sports Bar,Restaurant,Pizza Place


Cluster 5

In [32]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[0] + list(range(7, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
114,The Junction South,Café,Mexican Restaurant,Bar,Grocery Store,Arts & Crafts Store,Music Venue,Diner,Cajun / Creole Restaurant,Sandwich Place,Bookstore
174,Davisville North,Pizza Place,Breakfast Spot,Burger Joint,Food & Drink Shop,Hotel,Clothing Store,Park,Grocery Store,Sandwich Place,Electronics Store


end